## 2 The Data

In [363]:

import wrds
import pandas as pd
import datetime
import numpy as np


In [288]:
# Connect to WRDS
db = wrds.Connection()
db.create_pgpass_file()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [311]:
#---------------------------------------------
# WRDS Monthly World Indices
#---------------------------------------------

df = db.raw_sql("""
    SELECT * 
    FROM wrdsapps_windices.mwcountryreturns
    WHERE date BETWEEN '2002-01-01' AND '2024-12-31'
    AND country IN ('AUSTRALIA', 'FRANCE', 'GERMANY', 'JAPAN', 'SWITZERLAND', 'UNITED KINGDOM')
""")

df = df[["date", "country", "mportret", "currency"]]


#---------------------------------------------
# Risk Free Rate 
#---------------------------------------------
rf=db.raw_sql("""select  mcaldt,tmytm 
           from crsp.tfz_mth_rf            
            where kytreasnox = 2000001 
           and mcaldt>='2002-01-01'
            and mcaldt<='2024-12-31'""", date_cols=['mcaldt'])
rf['tmytm']=rf['tmytm']/12/100
rf=rf.rename(columns={ "mcaldt": "date","tmytm": "rf"})

#---------------------------------------------
# Value Weighted Index Returns US
#---------------------------------------------
rm=db.raw_sql("""select  date,vwretd from crsp.msi 
                where date>='2002-01-01' and date<='2024-12-31'
                """,date_cols=['date'])
rm = rm.rename(columns={'vwretd':'rm'})


#---------------------------------------------
# Convert date formats
#---------------------------------------------
rm['date'] = pd.to_datetime(rm['date'], format='%Y-%m-%d')
rf['date'] = pd.to_datetime(rf['date'], format='%Y-%m-%d')
rm['date'] = pd.to_datetime(rm['date']).dt.to_period('M').dt.to_timestamp()
rf['date'] = pd.to_datetime(rf['date']).dt.to_period('M').dt.to_timestamp()

#---------------------------------------------
# FX Rates
#---------------------------------------------
JPUS = pd.read_csv('EXJPUS.csv', sep=',')
SZUS = pd.read_csv('EXSZUS.csv', sep=',')
USEU = pd.read_csv('EXUSEU.csv', sep=',')
USAL = pd.read_csv('EXUSAL.csv', sep=',')
USUK = pd.read_csv('EXUSUK.csv', sep=',')

megre = pd.merge(SZUS, JPUS, on='observation_date', how='outer')
megre = pd.merge(megre, USEU, on='observation_date', how='outer')
megre = pd.merge(megre, USAL, on='observation_date', how='outer')
fx = pd.merge(megre, USUK, on='observation_date', how='outer')
fx = fx[(fx['observation_date'] >= '2002-02-01') & (fx['observation_date'] <= '2025-01-31')]
fx['EXSZUS'] = 1/fx['EXSZUS']
fx['EXJPUS'] = 1/fx['EXJPUS']
fx = fx.rename(columns={
    'observation_date': 'date',
    'EXSZUS': 'CHF',
    'EXJPUS': 'JPY',
    'EXUSEU': 'EUR',
    'EXUSAL': 'AUD',
    'EXUSUK': 'GBP'
})

fx['date'] = pd.to_datetime(fx['date'], format='%Y-%m-%d')
fx['date'] += datetime.timedelta(days=-1) # Adjusting to the last day of the month

# ---------------------------------------------
# Interbank Rates 3M
# ---------------------------------------------
rate_au = pd.read_csv("IR3TIB01AUM156N.csv", sep=',')
rate_ch = pd.read_csv("IR3TIB01CHM156N.csv", sep=',')
rate_de = pd.read_csv("IR3TIB01DEM156N.csv", sep=',')
rate_fr = pd.read_csv("IR3TIB01FRM156N.csv", sep=',')
rate_jp = pd.read_csv("IR3TIB01JPM156N.csv", sep=',')
rate_uk = pd.read_csv("IR3TIB01GBM156N.csv", sep=',')
rate_us = pd.read_csv("IR3TIB01USM156N.csv", sep=',')

rates = pd.merge(rate_au, rate_ch, on='observation_date', how='outer')
rates = pd.merge(rates, rate_de, on='observation_date', how='outer')
rates = pd.merge(rates, rate_fr, on='observation_date', how='outer')
rates = pd.merge(rates, rate_jp, on='observation_date', how='outer')
rates = pd.merge(rates, rate_uk, on='observation_date', how='outer')
rates = pd.merge(rates, rate_us, on='observation_date', how='outer')
rates = rates[(rates['observation_date'] >= '2002-02-01') & (rates['observation_date'] <= '2025-01-31')]
rates = rates.rename(columns={
    'observation_date': 'date',
    'IR3TIB01AUM156N': 'AUSTRALIA',
    'IR3TIB01CHM156N': 'SWITZERLAND',
    'IR3TIB01DEM156N': 'GERMANY',
    'IR3TIB01FRM156N': 'FRANCE',
    'IR3TIB01JPM156N': 'JAPAN',
    'IR3TIB01GBM156N': 'UNITED KINGDOM',
    'IR3TIB01USM156N': 'UNITED STATES'
})
#divide by 100 to get the rates in percentage
rates[['AUSTRALIA', 'SWITZERLAND', 'GERMANY', 'FRANCE', 'JAPAN', 'UNITED KINGDOM', 'UNITED STATES']] = rates[['AUSTRALIA', 'SWITZERLAND', 'GERMANY', 'FRANCE', 'JAPAN', 'UNITED KINGDOM', 'UNITED STATES']] / 100 /12

rates['date'] = pd.to_datetime(rates['date'], format='%Y-%m-%d')
rates['date'] += datetime.timedelta(days=-1)
print(rates.head())
#For Japan we miss 3 observations (for Japan we consider from 04/2002 - based on ED)

#---------------------------------------------
# Adjusting the dates
#---------------------------------------------
df['date'] = pd.to_datetime(df['date']).dt.to_period('M').dt.to_timestamp()
fx['date'] = pd.to_datetime(fx['date']).dt.to_period('M').dt.to_timestamp()
rates['date'] = pd.to_datetime(rates['date']).dt.to_period('M').dt.to_timestamp()

          date  AUSTRALIA  SWITZERLAND   GERMANY    FRANCE     JAPAN  \
505 2002-01-31   0.003592     0.001306  0.002798  0.002798       NaN   
506 2002-02-28   0.003717     0.001337  0.002826  0.002826       NaN   
507 2002-03-31   0.003825     0.001217  0.002839  0.002839  0.000083   
508 2002-04-30   0.004033     0.000970  0.002889  0.002889  0.000067   
509 2002-05-31   0.004225     0.000958  0.002887  0.002887  0.000075   

     UNITED KINGDOM  UNITED STATES  
505        0.003377       0.001517  
506        0.003442       0.001592  
507        0.003478       0.001558  
508        0.003461       0.001517  
509        0.003481       0.001508  


In [312]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
fx['date'] = pd.to_datetime(fx['date'], format='%Y-%m-%d')
rates['date'] = pd.to_datetime(rates['date'], format='%Y-%m-%d')

df = pd.merge(df, fx, on='date',  how='left')
df['fx'] = df.apply(lambda row: row[row['currency']], axis=1)
df = df.drop(columns=['CHF', 'JPY', 'EUR', 'AUD', 'GBP'])

df = pd.merge(df, rf, on='date', how='left')
df = pd.merge(df, rm, on='date', how='left')

In [313]:
df = pd.merge(df, rates, on='date',  how='left')
df['rates'] = df.apply(lambda row: row[row['country']], axis=1)
df = df.drop(columns=['AUSTRALIA', 'SWITZERLAND', 'GERMANY', 'FRANCE', 'JAPAN', 'UNITED KINGDOM', 'UNITED STATES'])

## EX 3 A

In [362]:
df['fx_t+1'] = df['fx'].shift(-1)
df['returns_USD'] = (1 + df['mportret']) * (1 + (df['fx_t+1'] - df['fx'])/df['fx']) - 1
df['rates_USD'] = (1 + df['rates']) * (1 + (df['fx_t+1'] - df['fx'])/df['fx']) - 1
display(df.head())

,date,country,mportret,currency,fx,rf,rm,rates,fx_t+1,returns_USD,X,hedged_return,rates_USD
0,2002-01-01,AUSTRALIA,0.027358,AUD,0.5128,0.001408,-0.015966,0.003592,0.5256,0.053002,0.027234,0.025767,0.028642
1,2002-02-01,AUSTRALIA,-0.001832,AUD,0.5256,0.001444,-0.0217,0.003717,0.5352,0.016399,0.020606,-0.004206,0.022049
2,2002-03-01,AUSTRALIA,0.005199,AUD,0.5352,0.001425,0.044698,0.003825,0.5498,0.03262,0.029784,0.002836,0.031209
3,2002-04-01,AUSTRALIA,-0.014302,AUD,0.5498,0.001461,-0.0496,0.004033,0.5682,0.018686,0.036174,-0.017488,0.037635
4,2002-05-01,AUSTRALIA,-0.000204,AUD,0.5682,0.001408,-0.01051,0.004225,0.5538,-0.025542,-0.022633,-0.002909,-0.021225


## 3 B

In [315]:
# Calculate currency-hedged component
df['X'] = (df['fx_t+1'] / df['fx']) * (1 + df['rates']) - (1 + df['rf'])

# Calculate hedged index return
df['hedged_return'] = df['returns_USD'] - df['X']
display(df.head())

,date,country,mportret,currency,fx,rf,rm,rates,fx_t+1,returns_USD,X,hedged_return
0,2002-01-01,AUSTRALIA,0.027358,AUD,0.5128,0.001408,-0.015966,0.003592,0.5256,0.053002,0.027234,0.025767
1,2002-02-01,AUSTRALIA,-0.001832,AUD,0.5256,0.001444,-0.0217,0.003717,0.5352,0.016399,0.020606,-0.004206
2,2002-03-01,AUSTRALIA,0.005199,AUD,0.5352,0.001425,0.044698,0.003825,0.5498,0.03262,0.029784,0.002836
3,2002-04-01,AUSTRALIA,-0.014302,AUD,0.5498,0.001461,-0.0496,0.004033,0.5682,0.018686,0.036174,-0.017488
4,2002-05-01,AUSTRALIA,-0.000204,AUD,0.5682,0.001408,-0.01051,0.004225,0.5538,-0.025542,-0.022633,-0.002909


## 3C

In [ ]:
#-----------------------------------------------
# Unhedged Index Returns
#-----------------------------------------------

#-----------------------------------------------
#equaly weighted index returns
#-----------------------------------------------
results_eq = df.groupby('date').returns_USD
mean_eq = results_eq.mean()*12
variance_eq = results_eq.variance()*np.sqrt(12)
sharpe_eq = (mean_eq - 12*df['rates_USD'].mean())/np.sqrt(variance_eq)

#-----------------------------------------------
# Risk parity index returns
#-----------------------------------------------
result = df.groupby('country').returns_USD.rolling(60, min_periods=1).std().reset_index().rename(columns={'returns_USD': 'std'})
result['date'] = df['date']
result['weights'] = 1 / result['std']
result['weights'] = result['weights'] / (pd.concat([result.groupby('date').weights.sum().reset_index()['weights']] * 6, ignore_index=True))
result.drop(columns=['level_1', 'std'], inplace=True)
merged = pd.merge(df, result, on=['date', 'country'])
merged['weighted_return'] = merged['returns_USD'] * merged['weights']
results_rp = merged.groupby('date').weighted_return.sum()
mean_rp = results_rp.mean()*12
variance_rp = results_rp.variance()*np.sqrt(12)
sharpe_rp = (mean_rp - 12*df['rates_USD'].mean())/np.sqrt(variance_rp)

#-----------------------------------------------
# Mean variance portfolio returns
#-----------------------------------------------
gamma = 1.0
grouped = df.groupby("country").returns_USD.rolling(60, min_periods=1)
result = grouped.cov(grouped)
print(result)




AttributeError: 'SeriesGroupBy' object has no attribute 'variance'

In [317]:
#-----------------------------------------------
# Hedged Index Returns
#-----------------------------------------------
print(df.groupby('date').hedged_return.mean())

date
2002-01-01   -0.005432
2002-02-01   -0.002783
2002-03-01    0.041548
2002-04-01   -0.018388
2002-05-01   -0.005622
                ...   
2024-08-01    0.006897
2024-09-01    0.005749
2024-10-01   -0.017372
2024-11-01    0.009985
2024-12-01    0.005049
Name: hedged_return, Length: 276, dtype: Float64


In [ ]:
    #-----------------------------
    # Raw Returns
    #-----------------------------
    riskfree=data["rf"].mean()

    stocks_mean=data_t["Stocks"].mean()*12
    stocks_vol=data_t["Stocks"].std()*np.sqrt(12)

    #-----------------------------
    # Risk Parity PORTFOLIO
    #-----------------------------
    print('RP Weight Stocks:', 1/stocks_vol/(1/bonds_vol+1/stocks_vol))
    data_t["Risk_Parity"]=(1/bonds_vol*data_t["Bonds"]+1/stocks_vol*data_t["Stocks"])/(1/bonds_vol+1/stocks_vol)
    RP_mean=data_t["Risk_Parity"].mean()*12
    RP_vol=data_t["Risk_Parity"].std()*np.sqrt(12)
    k=mix_vol/RP_vol
    data_t["Risk_Parity_levered"]=k*data_t["Risk_Parity"]+(1-k)*data_t["rf"]

    rp_mean=data_t["Risk_Parity"].mean()*12
    rp_vol=data_t["Risk_Parity"].std()*np.sqrt(12)

    rp_levered_mean=data_t["Risk_Parity_levered"].mean()*12
    rp_levered_vol=data_t["Risk_Parity_levered"].std()*np.sqrt(12)

    rp_sr=(rp_mean-riskfree*12)/rp_vol
    rp_levered_sr=(rp_levered_mean-riskfree*12)/rp_levered_vol